# Modèle BERT pour la Classification de Sentiments

Ce notebook implémente un modèle BERT (Bidirectional Encoder Representations from Transformers) pour la classification de sentiments des tweets.

**Objectif**: Fine-tuner un modèle BERT pré-entraîné sur notre dataset de tweets pour la classification binaire (positif/négatif).

**Approche**: Utilisation de `TFBertForSequenceClassification` de Hugging Face avec tokenisation BERT et préparation des input_ids et attention_masks.

## 1. Imports et Configuration

In [ ]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.tensorflow
import time
import os
from datetime import datetime

import tensorflow as tf
from transformers import BertTokenizer, TFBertForSequenceClassification
from transformers import DataCollatorWithPadding

from sklearn.metrics import (
    accuracy_score,
    precision_score,
    recall_score,
    f1_score,
    roc_auc_score,
    confusion_matrix,
    classification_report
)

import matplotlib.pyplot as plt
import seaborn as sns

# Configuration
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 100)
np.random.seed(42)
tf.random.set_seed(42)

print(f"TensorFlow version: {tf.__version__}")
print(f"GPU disponible: {tf.config.list_physical_devices('GPU')}")

## 2. Configuration MLFlow

In [ ]:
# Configuration MLFlow
mlflow.set_tracking_uri("http://127.0.0.1:5001")
mlflow.set_experiment("air-paradis-sentiment-analysis")

def log_metrics_to_mlflow(model_name, metrics_dict, model=None, training_time=None, prediction_time=None):
    """
    Fonction pour logger les métriques dans MLFlow de manière standardisée.
    
    Args:
        model_name: Nom du modèle
        metrics_dict: Dictionnaire contenant les métriques
        model: Modèle TensorFlow (optionnel)
        training_time: Temps d'entraînement en secondes
        prediction_time: Temps de prédiction en secondes
    """
    # Log des métriques
    for metric_name, metric_value in metrics_dict.items():
        mlflow.log_metric(metric_name, metric_value)
    
    # Log des temps
    if training_time:
        mlflow.log_metric("training_time_seconds", training_time)
    if prediction_time:
        mlflow.log_metric("prediction_time_seconds", prediction_time)
    
    # Log du modèle
    if model:
        mlflow.tensorflow.log_model(model, "model")
    
    print(f"✓ Métriques loggées dans MLFlow pour {model_name}")

## 3. Chargement des Données

Pour BERT, nous utilisons les données **brutes** (non prétraitées) car BERT a son propre tokenizer qui gère la tokenisation, la casse, et les caractères spéciaux de manière optimale.

In [ ]:
# Chargement des données originales (avant preprocessing NLTK)
# BERT fonctionne mieux avec le texte brut car il a son propre tokenizer
print("Chargement des données...")

# Pour l'entraînement BERT, nous allons utiliser un subset plus petit pour des raisons de temps
# BERT est très coûteux en ressources
SAMPLE_SIZE = 50000  # 50k tweets par set (ajustable selon vos ressources)

# Charger les données originales
train_df = pd.read_csv('../data/processed/train_lemmatized.csv').sample(n=SAMPLE_SIZE, random_state=42)
val_df = pd.read_csv('../data/processed/val_lemmatized.csv').sample(n=int(SAMPLE_SIZE * 0.2), random_state=42)
test_df = pd.read_csv('../data/processed/test_lemmatized.csv').sample(n=int(SAMPLE_SIZE * 0.2), random_state=42)

# Pour BERT, nous utilisons le texte (même prétraité, BERT s'en sortira bien)
# Note: Idéalement, on utiliserait le texte complètement brut, mais le texte lemmatisé fonctionne aussi

print(f"Train: {len(train_df)} tweets")
print(f"Validation: {len(val_df)} tweets")
print(f"Test: {len(test_df)} tweets")

# Vérification
print("\nExemple de tweet:")
print(train_df['text'].iloc[0])
print(f"\nSentiment: {train_df['sentiment'].iloc[0]}")

## 4. Tokenisation BERT

BERT nécessite une tokenisation spécifique qui:
- Convertit le texte en tokens BERT (WordPiece)
- Ajoute les tokens spéciaux [CLS] et [SEP]
- Crée les input_ids (IDs des tokens)
- Crée les attention_masks (masque pour indiquer les vrais tokens vs padding)

In [ ]:
# Initialisation du tokenizer BERT
print("Initialisation du tokenizer BERT...")
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Configuration de la longueur maximale des séquences
MAX_LENGTH = 128  # Longueur maximale des tweets (ajustable)

def tokenize_data(texts, labels, max_length=MAX_LENGTH):
    """
    Tokenise les textes avec le tokenizer BERT.
    
    Returns:
        input_ids: IDs des tokens
        attention_masks: Masques d'attention
        labels: Labels
    """
    encodings = tokenizer(
        texts.tolist(),
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='tf'
    )
    
    return encodings['input_ids'], encodings['attention_mask'], tf.constant(labels.values)

print("Tokenisation des données...")
start_time = time.time()

# Tokenisation
train_input_ids, train_attention_masks, train_labels = tokenize_data(train_df['text'], train_df['sentiment'])
val_input_ids, val_attention_masks, val_labels = tokenize_data(val_df['text'], val_df['sentiment'])
test_input_ids, test_attention_masks, test_labels = tokenize_data(test_df['text'], test_df['sentiment'])

tokenization_time = time.time() - start_time
print(f"✓ Tokenisation terminée en {tokenization_time:.2f}s")

# Vérification des dimensions
print(f"\nDimensions:")
print(f"Train input_ids: {train_input_ids.shape}")
print(f"Train attention_masks: {train_attention_masks.shape}")
print(f"Train labels: {train_labels.shape}")

# Exemple de tokenisation
print(f"\nExemple de tokenisation:")
print(f"Texte original: {train_df['text'].iloc[0]}")
print(f"Input IDs: {train_input_ids[0][:20]}...")  # Premiers 20 tokens
print(f"Attention mask: {train_attention_masks[0][:20]}...")  # Premiers 20 masques

## 5. Création des Datasets TensorFlow

In [ ]:
BATCH_SIZE = 16  # Batch size pour BERT (ajustable selon la mémoire GPU)

# Création des datasets
train_dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': train_input_ids,
    'attention_mask': train_attention_masks
}, train_labels))

val_dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': val_input_ids,
    'attention_mask': val_attention_masks
}, val_labels))

test_dataset = tf.data.Dataset.from_tensor_slices(({
    'input_ids': test_input_ids,
    'attention_mask': test_attention_masks
}, test_labels))

# Configuration des datasets pour l'entraînement
train_dataset = train_dataset.shuffle(10000).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val_dataset = val_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
test_dataset = test_dataset.batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)

print(f"✓ Datasets créés avec batch_size={BATCH_SIZE}")

## 6. Construction du Modèle BERT

Nous utilisons `TFBertForSequenceClassification` qui est un modèle BERT pré-entraîné avec une couche de classification ajoutée.

In [ ]:
def build_bert_model():
    """
    Construit un modèle BERT pour la classification binaire.
    """
    from transformers import create_optimizer
    
    model = TFBertForSequenceClassification.from_pretrained(
        'bert-base-uncased',
        num_labels=2,  # Classification binaire
        use_safetensors=False  # Évite les problèmes de compatibilité
    )
    
    # Note: Pas besoin de compiler manuellement pour BERT
    # Le modèle est déjà configuré pour l'entraînement
    
    return model

print("Construction du modèle BERT...")
bert_model = build_bert_model()

print("\n" + "="*80)
print("ARCHITECTURE DU MODÈLE BERT")
print("="*80)
print(f"✓ Modèle BERT chargé: bert-base-uncased")
print(f"✓ Nombre de labels: 2 (classification binaire)")
print(f"✓ Paramètres: ~110M")
print("="*80)

## 7. Entraînement du Modèle

In [ ]:
# Configuration de l'entraînement
EPOCHS = 3  # 3-4 epochs sont généralement suffisants pour BERT
LEARNING_RATE = 2e-5

# Création de l'optimizer et compilation
from transformers import create_optimizer

# Calculer le nombre de steps
num_train_steps = len(train_dataset) * EPOCHS

optimizer, lr_schedule = create_optimizer(
    init_lr=LEARNING_RATE,
    num_train_steps=num_train_steps,
    num_warmup_steps=int(num_train_steps * 0.1),  # 10% de warmup
)

# Compiler le modèle avec le bon optimizer
bert_model.compile(
    optimizer=optimizer,
    loss=tf.keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    metrics=['accuracy']
)

# Début du run MLFlow
with mlflow.start_run(run_name="BERT-base-uncased"):
    # Log des hyperparamètres
    mlflow.log_param("model_type", "BERT")
    mlflow.log_param("pretrained_model", "bert-base-uncased")
    mlflow.log_param("max_length", MAX_LENGTH)
    mlflow.log_param("batch_size", BATCH_SIZE)
    mlflow.log_param("epochs", EPOCHS)
    mlflow.log_param("learning_rate", LEARNING_RATE)
    mlflow.log_param("train_samples", len(train_df))
    mlflow.log_param("val_samples", len(val_df))
    
    print("\nDébut de l'entraînement...")
    print("="*80)
    print("Note: Entraînement de 3 epochs sans early stopping")
    print("="*80)
    
    start_time = time.time()
    
    # Entraînement (sans callbacks pour éviter les problèmes de compatibilité)
    history = bert_model.fit(
        train_dataset,
        validation_data=val_dataset,
        epochs=EPOCHS,
        verbose=1
    )
    
    training_time = time.time() - start_time
    
    print("="*80)
    print(f"✓ Entraînement terminé en {training_time:.2f}s ({training_time/60:.2f} minutes)")
    
    # Log du temps d'entraînement
    mlflow.log_metric("training_time_seconds", training_time)
    mlflow.log_metric("training_time_minutes", training_time/60)

## 8. Visualisation de l'Entraînement

In [ ]:
# Graphique d'entraînement
fig, axes = plt.subplots(1, 2, figsize=(15, 5))

# Loss
axes[0].plot(history.history['loss'], label='Train Loss', marker='o')
axes[0].plot(history.history['val_loss'], label='Validation Loss', marker='o')
axes[0].set_title('Evolution de la Loss', fontsize=14, fontweight='bold')
axes[0].set_xlabel('Epoch')
axes[0].set_ylabel('Loss')
axes[0].legend()
axes[0].grid(True, alpha=0.3)

# Accuracy
axes[1].plot(history.history['accuracy'], label='Train Accuracy', marker='o')
axes[1].plot(history.history['val_accuracy'], label='Validation Accuracy', marker='o')
axes[1].set_title('Evolution de l\'Accuracy', fontsize=14, fontweight='bold')
axes[1].set_xlabel('Epoch')
axes[1].set_ylabel('Accuracy')
axes[1].legend()
axes[1].grid(True, alpha=0.3)

plt.tight_layout()
plt.savefig('bert_training_history.png', dpi=300, bbox_inches='tight')
plt.show()

# Log dans MLFlow
with mlflow.start_run(run_id=mlflow.active_run().info.run_id):
    mlflow.log_artifact('bert_training_history.png')

## 9. Évaluation sur le Set de Validation

In [ ]:
print("Évaluation sur le set de validation...")

# Prédictions
start_time = time.time()
val_predictions = bert_model.predict(val_dataset)
prediction_time = time.time() - start_time

# Extraction des logits et conversion en classes
val_logits = val_predictions.logits
val_pred_classes = np.argmax(val_logits, axis=1)

# Probabilités (pour ROC-AUC)
val_probabilities = tf.nn.softmax(val_logits, axis=1).numpy()[:, 1]

# Calcul des métriques
val_metrics = {
    'val_accuracy': accuracy_score(val_labels.numpy(), val_pred_classes),
    'val_precision': precision_score(val_labels.numpy(), val_pred_classes),
    'val_recall': recall_score(val_labels.numpy(), val_pred_classes),
    'val_f1_score': f1_score(val_labels.numpy(), val_pred_classes),
    'val_roc_auc': roc_auc_score(val_labels.numpy(), val_probabilities)
}

print("\n" + "="*80)
print("RÉSULTATS SUR LE SET DE VALIDATION")
print("="*80)
print(f"Accuracy:  {val_metrics['val_accuracy']:.4f}")
print(f"Precision: {val_metrics['val_precision']:.4f}")
print(f"Recall:    {val_metrics['val_recall']:.4f}")
print(f"F1-Score:  {val_metrics['val_f1_score']:.4f}")
print(f"ROC-AUC:   {val_metrics['val_roc_auc']:.4f}")
print(f"\nTemps de prédiction: {prediction_time:.2f}s")
print("="*80)

# Log dans MLFlow
with mlflow.start_run(run_id=mlflow.active_run().info.run_id):
    log_metrics_to_mlflow(
        "BERT",
        val_metrics,
        prediction_time=prediction_time
    )

## 10. Matrice de Confusion

In [ ]:
# Matrice de confusion
cm = confusion_matrix(val_labels.numpy(), val_pred_classes)

plt.figure(figsize=(8, 6))
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', cbar=True)
plt.title('Matrice de Confusion - BERT', fontsize=14, fontweight='bold')
plt.ylabel('Vraie Classe')
plt.xlabel('Classe Prédite')
plt.xticks([0.5, 1.5], ['Négatif (0)', 'Positif (1)'])
plt.yticks([0.5, 1.5], ['Négatif (0)', 'Positif (1)'])
plt.tight_layout()
plt.savefig('bert_confusion_matrix.png', dpi=300, bbox_inches='tight')
plt.show()

# Log dans MLFlow
with mlflow.start_run(run_id=mlflow.active_run().info.run_id):
    mlflow.log_artifact('bert_confusion_matrix.png')

# Rapport de classification détaillé
print("\n" + "="*80)
print("RAPPORT DE CLASSIFICATION")
print("="*80)
print(classification_report(val_labels.numpy(), val_pred_classes, 
                          target_names=['Négatif', 'Positif']))
print("="*80)

## 11. Évaluation Finale sur le Set de Test

In [ ]:
print("Évaluation finale sur le set de test...")

# Prédictions sur le test
test_predictions = bert_model.predict(test_dataset)
test_logits = test_predictions.logits
test_pred_classes = np.argmax(test_logits, axis=1)
test_probabilities = tf.nn.softmax(test_logits, axis=1).numpy()[:, 1]

# Métriques finales
test_metrics = {
    'test_accuracy': accuracy_score(test_labels.numpy(), test_pred_classes),
    'test_precision': precision_score(test_labels.numpy(), test_pred_classes),
    'test_recall': recall_score(test_labels.numpy(), test_pred_classes),
    'test_f1_score': f1_score(test_labels.numpy(), test_pred_classes),
    'test_roc_auc': roc_auc_score(test_labels.numpy(), test_probabilities)
}

print("\n" + "="*80)
print("RÉSULTATS FINAUX SUR LE SET DE TEST")
print("="*80)
print(f"Accuracy:  {test_metrics['test_accuracy']:.4f}")
print(f"Precision: {test_metrics['test_precision']:.4f}")
print(f"Recall:    {test_metrics['test_recall']:.4f}")
print(f"F1-Score:  {test_metrics['test_f1_score']:.4f}")
print(f"ROC-AUC:   {test_metrics['test_roc_auc']:.4f}")
print("="*80)

# Log dans MLFlow
with mlflow.start_run(run_id=mlflow.active_run().info.run_id):
    for metric_name, metric_value in test_metrics.items():
        mlflow.log_metric(metric_name, metric_value)

## 12. Sauvegarde du Modèle

In [ ]:
# Créer le dossier si nécessaire
os.makedirs('../models', exist_ok=True)

# Sauvegarde du modèle complet
model_path = '../models/bert_sentiment_model'
bert_model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

print(f"✓ Modèle BERT sauvegardé dans: {model_path}")

# Log dans MLFlow
with mlflow.start_run(run_id=mlflow.active_run().info.run_id):
    mlflow.log_artifact(model_path)

## 13. Test de Prédiction sur des Exemples

In [ ]:
def predict_sentiment(text, model, tokenizer, max_length=MAX_LENGTH):
    """
    Prédit le sentiment d'un texte.
    
    Returns:
        sentiment: 'Positif' ou 'Négatif'
        probability: Probabilité de la classe prédite
    """
    # Tokenisation
    encoding = tokenizer(
        text,
        truncation=True,
        padding='max_length',
        max_length=max_length,
        return_tensors='tf'
    )
    
    # Prédiction
    outputs = model(encoding)
    logits = outputs.logits
    probabilities = tf.nn.softmax(logits, axis=1).numpy()[0]
    
    predicted_class = np.argmax(probabilities)
    sentiment = 'Positif' if predicted_class == 1 else 'Négatif'
    confidence = probabilities[predicted_class]
    
    return sentiment, confidence

# Tests sur des exemples
test_examples = [
    "This flight was amazing! Best experience ever!",
    "Terrible service, never flying with them again",
    "The staff was friendly and helpful",
    "Delayed for 5 hours, worst airline",
    "Good value for money"
]

print("\n" + "="*80)
print("TESTS DE PRÉDICTION")
print("="*80)

for text in test_examples:
    sentiment, confidence = predict_sentiment(text, bert_model, tokenizer)
    print(f"\nTexte: {text}")
    print(f"Sentiment prédit: {sentiment} (confiance: {confidence:.2%})")

print("\n" + "="*80)

## 14. Comparaison avec les Autres Modèles

Cette section sera complétée après avoir exécuté tous les notebooks pour comparer:
- Modèle simple (Logistic Regression)
- Modèles avancés (Bi-LSTM, CNN avec Word2Vec/GloVe)
- Modèle BERT

Critères de comparaison:
1. **Performance**: Accuracy, F1-Score, ROC-AUC
2. **Temps d'entraînement**: Combien de temps pour entraîner?
3. **Temps de prédiction**: Vitesse d'inférence
4. **Complexité**: Nombre de paramètres, ressources nécessaires
5. **Facilité de déploiement**: Taille du modèle, dépendances

In [ ]:
# Cette cellule sera utilisée pour créer un tableau comparatif final
# après avoir exécuté tous les notebooks

print("\n" + "="*80)
print("RÉSUMÉ DES PERFORMANCES - BERT")
print("="*80)
print(f"\nValidation Set:")
for metric, value in val_metrics.items():
    print(f"  {metric}: {value:.4f}")

print(f"\nTest Set:")
for metric, value in test_metrics.items():
    print(f"  {metric}: {value:.4f}")

print(f"\nTemps d'entraînement: {training_time/60:.2f} minutes")
print(f"Temps de prédiction: {prediction_time:.2f} secondes")
print("="*80)

## Conclusion

Ce notebook a implémenté un modèle BERT pour la classification de sentiments de tweets.

**Points clés:**
- Utilisation de `bert-base-uncased` pré-entraîné
- Fine-tuning sur notre dataset de tweets
- Préparation appropriée des données (input_ids, attention_masks)
- Tracking complet avec MLFlow

**Prochaines étapes:**
1. Comparer BERT avec les modèles précédents
2. Choisir le meilleur modèle pour le déploiement
3. Créer l'API pour le modèle sélectionné
4. Déployer sur le Cloud
5. Mettre en place le monitoring avec Azure Application Insights